# Lab 3: Supervised Learning - Classification Algorithms

### Aidan Barbieux and Eric Inman

In [19]:
import numpy as np
import pandas as pd
import math

## C4.5 Algorithm

### Entropy Calculation

In [37]:
# takes the column of classes from dataset and computes information entropy
def entropy(classCol):
    vals = classCol.value_counts()
    size = len(classCol)
    entropy=0
    for v in vals:
        entropy -= (v/size) * math.log(v/size,2)
    return entropy

### Tree class

In [ ]:
class Tree :
    def __init__(self):
        self.children = {}
        self.label = ''

### C4.5

In [ ]:
def c45(data, attrs, tree, thresh):
    

## Data

In [35]:
astretch = pd.read_csv("adult-stretch.csv")
astretch = astretch.drop([0,1], axis=0)
entropy(astretch['Inflated'])

T 12 :  -0.44217935649972373
F 8 :  -0.5287712379549449


-0.9709505944546686